In [2]:
pip install git+https://github.com/huggingface/transformers@21fac7abba2a37fae86106f87fcf9974fd1e3830 accelerate flash_attn

  Cloning https://github.com/huggingface/transformers (to revision 21fac7abba2a37fae86106f87fcf9974fd1e3830) to c:\users\acer\appdata\local\temp\pip-req-build-e4621up0
  Resolved https://github.com/huggingface/transformers to commit 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached flash_attn-2.7.2.post1.tar.gz (3.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\acer\AppData\Local\Temp\pip-req-build-e4621up0'
  Running command git rev-parse -q --verify 'sha^21fac7abba2a37fae86106f87fcf9974fd1e3830'
  Running command git fetch -q https://github.com/huggingface/transformers 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Running command git checkout -q 21fac7abba2a37fae86106f87fcf9974fd1e3830
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      
      
      torch.__version__  = 2.5.1+cpu
      
      
      C:\Users\acer\AppData\Local\Temp\pip-install-f28qzyr0\flash-attn_41bd57df1a1f4b36bad49fc6aac24e1a\setup.py:99: UserWarning: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from http

In [3]:
# It's highly recommanded to use `[decord]` feature for faster video loading.
pip install qwen-vl-utils[decord]

SyntaxError: invalid syntax (1826490617.py, line 2)

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

In [ ]:
model_name = "Qwen/Qwen2-VL-2B-Instruct"

In [ ]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2",
    device_map="auto"
)

In [ ]:
processor = AutoProcessor.from_pretrained(model_name)

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
              "type": "video",
              "video": "/content/test.mp4",
              "max_pixels": 360*420,
              "fps": 1.0,
            },
            {
                "type": "text",
                "text": "Can you explain the video in brief?"
            }
        ],
    }
]

In [ ]:
messages

In [ ]:
text = processor.apply_chat_template(
    messages, tokenize = False, add_generation_prompt = True
)

In [ ]:
text

In [ ]:
image_inputs, video_inputs = process_vision_info(messages)

In [ ]:
inputs = processor(
    text = [text],
    images = image_inputs,
    videos = video_inputs,
    padding = True,
    return_tensors = "pt"
)

In [ ]:
inputs = inputs.to("cuda")

In [ ]:
generated_ids = model.generate(**inputs, max_new_tokens=512)